### Session 13 Assignment 1

#### Predicting Survival in the Titanic Data Set
#### We will be using a decision tree to make predictions about the Titanic data set from Kaggle. This data set provides information on the Titanic passengers and can be used to predict whether a passenger survived or not.

#### Loading Data and modules

In [5]:
# Importing module and assigning alias for them.
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

#### Url= https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv

In [6]:
# loading a dataset from URL into a pandas dataframe titanic
titanic = pd.read_csv("https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### You use only Pclass, Sex, Age, SibSp (Siblings aboard), Parch (Parents/children aboard), and Fare to predict whether a passenger survived.

In [7]:
# Checking unique values in column Pclass, Sex, Age, SibSp,Parch and Fare
Pclass = set(titanic['Pclass'])
SEX = set(titanic['Sex'])
sibsp = set(titanic['SibSp'])
parch = set(titanic['Parch'])
print("PCLASS VALUES:",Pclass)
print("SEX VALUES:",SEX)
print("SIBSP VALUES:",sibsp)
print("PARCH VALUES:",parch)

PCLASS VALUES: {1, 2, 3}
SEX VALUES: {'female', 'male'}
SIBSP VALUES: {0, 1, 2, 3, 4, 5, 8}
PARCH VALUES: {0, 1, 2, 3, 4, 5, 6}


In [8]:
# Filling NAN values with median of that column
titanic['Pclass'].fillna((titanic['Pclass'].median()),inplace=True)
titanic['SibSp'].fillna((titanic['SibSp'].median()),inplace=True)
titanic['Parch'].fillna((titanic['Parch'].median()),inplace=True)
titanic['Age'].fillna((titanic['Age'].median()),inplace=True)
titanic['Fare'].fillna((titanic['Fare'].median()),inplace=True)

In [10]:
# To divide input in X and y axis
X = titanic.iloc[:, [2,4,5,6,7,9]].values
Y = titanic.iloc[:, 1].values
print("X-DATASET:\n",X)
print("Y-DATASET:\n",Y)

X-DATASET:
 [[3 'male' 22.0 1 0 7.25]
 [1 'female' 38.0 1 0 71.2833]
 [3 'female' 26.0 0 0 7.925]
 ...
 [3 'female' 28.0 1 2 23.45]
 [1 'male' 26.0 0 0 30.0]
 [3 'male' 32.0 0 0 7.75]]
Y-DATASET:
 [0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 

In [11]:
# Encoding categorical data
labelencoder = LabelEncoder()
X[:, 1] = labelencoder.fit_transform(X[:, 1])
onehotencoder = OneHotEncoder(categorical_features = [2])
X = onehotencoder.fit_transform(X).toarray()
print("X-DATASET AFTER ONE-HOT encoding of categorical data:\n",X)

X-DATASET AFTER ONE-HOT encoding of categorical data:
 [[ 0.      0.      0.     ...  1.      0.      7.25  ]
 [ 0.      0.      0.     ...  1.      0.     71.2833]
 [ 0.      0.      0.     ...  0.      0.      7.925 ]
 ...
 [ 0.      0.      0.     ...  1.      2.     23.45  ]
 [ 0.      0.      0.     ...  0.      0.     30.    ]
 [ 0.      0.      0.     ...  0.      0.      7.75  ]]


C:\Users\Swami\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Swami\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [12]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
print("X-TRAINING DATA:\n",X_train)
print("X-TEST DATA:\n",X_test)
print("Y-TRAINING DATA:\n",y_train)
print("Y-TEST DATA:\n",y_test)

X-TRAINING DATA:
 [[ 0.      0.      0.     ...  0.      2.     15.2458]
 [ 0.      0.      0.     ...  0.      0.     10.5   ]
 [ 0.      0.      0.     ...  1.      1.     37.0042]
 ...
 [ 0.      0.      0.     ...  0.      0.      7.7333]
 [ 0.      0.      0.     ...  1.      0.     17.4   ]
 [ 0.      0.      0.     ...  1.      1.     39.    ]]
X-TEST DATA:
 [[  0.       0.       0.     ...   0.       0.      14.4583]
 [  0.       0.       0.     ...   0.       0.       7.55  ]
 [  0.       0.       0.     ...   4.       1.      29.125 ]
 ...
 [  0.       0.       0.     ...   1.       0.     113.275 ]
 [  0.       0.       0.     ...   0.       0.       7.8542]
 [  0.       0.       0.     ...   0.       0.       8.05  ]]
Y-TRAINING DATA:
 [0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0
 0 1 0 1 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0
 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0
 1 0 1 1 1 0 0 0 

In [13]:
# Feature Scaling X data set
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print("X-TRAINING DATA AFTER FEATURE SCALING:\n",X_train)
print("X-TEST DATA AFTER FEATURE SCALING:\n",X_test)

X-TRAINING DATA AFTER FEATURE SCALING:
 [[-0.08409599 -0.09964476 -0.10660036 ... -0.46445234  1.95926403
  -0.33167904]
 [-0.08409599 -0.09964476 -0.10660036 ... -0.46445234 -0.47741019
  -0.42640542]
 [-0.08409599 -0.09964476 -0.10660036 ...  0.41270964  0.74092692
   0.10261958]
 ...
 [-0.08409599 -0.09964476 -0.10660036 ... -0.46445234 -0.47741019
  -0.48162887]
 [-0.08409599 -0.09964476 -0.10660036 ...  0.41270964 -0.47741019
  -0.28868112]
 [-0.08409599 -0.09964476 -0.10660036 ...  0.41270964  0.74092692
   0.14245584]]
X-TEST DATA AFTER FEATURE SCALING:
 [[-0.08409599 -0.09964476 -0.10660036 ... -0.46445234 -0.47741019
  -0.34739758]
 [-0.08409599 -0.09964476 -0.10660036 ... -0.46445234 -0.47741019
  -0.48528755]
 [-0.08409599 -0.09964476 -0.10660036 ...  3.04419557  0.74092692
  -0.0546496 ]
 ...
 [-0.08409599 -0.09964476 -0.10660036 ...  0.41270964 -0.47741019
   1.62498811]
 [-0.08409599 -0.09964476 -0.10660036 ... -0.46445234 -0.47741019
  -0.4792157 ]
 [-0.08409599 -0.09964

In [14]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

C:\Users\Swami\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0], dtype=int64)

In [16]:
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("CONFUSIONMATRIX:\n",cm)

CONFUSIONMATRIX:
 [[95 15]
 [19 50]]


In [17]:
print ("MODEL ACCURACY:",metrics.accuracy_score(y_test,y_pred))

MODEL ACCURACY: 0.8100558659217877
